In [1]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env

True

In [2]:
from langchain_openai import OpenAIEmbeddings
from servicenow import ServiceNowLoader
from langchain.vectorstores import FAISS

table = "kb_knowledge"
query = "short_descriptionLIKEsmartcool"
fields = "short_description,text"
loader = ServiceNowLoader(table, query, fields)

In [3]:
docs = loader.lazy_load()
docs

<generator object ServiceNowLoader.lazy_load at 0x108f2eac0>

In [4]:
docarr = []
for doc in docs:
    print(type(doc))
    print(doc.page_content)
    print("-----")
    docarr.append(doc)

<class 'langchain_core.documents.base.Document'>

short_description:Changing the freezer temparature on the SmartCool refridgerator
text:


From the Home screen, swipe left.Touch Fridge Manager.Touch the freezer temperature.There are 5 preset temperatures. Select a desired temperature.To change the temperature manually, use the arrow buttons.Touch OK to finish.



 
 

-----
<class 'langchain_core.documents.base.Document'>

short_description:Viewing inside the SmartCool refrigerator using the			SmartView camera from a mobile device
text:SmartCool refrigerators can be paired with the companion mobile app to access a number of useful features like remotely controlling your fridge or freezer temperature, or to view inside the fridge. 
 
1
Open SmartCool app on your mobile device.
note:SmartCool app can be downloaded from Google Play Store.
2
Make sure your SmartCool fridge is connected to the SmartCool App.Swipe left to access the Menu page.
3
Touch View Inside.
4
SmartCool App will displ

In [5]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docarr, embeddings)

In [6]:
retriever = vectorstore.as_retriever()

In [7]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10ad6e5d0>, search_kwargs={})

In [8]:
ret  = retriever.invoke("what types of things did the author want to build?")

In [9]:
ret

[Document(metadata={'line_number': 3, 'source': 'https://dev228679.service-now.com'}, page_content='\nshort_description:Connecting the SmartCool Refrigerator to Wi-Fi\t\t\ntext:\n\n\nFrom the Home screen, swipe left.Touch Settings.Touch Wi-Fi.Slide Wi-Fi to On. SmartCool will begin scanning for wireless networks.Touch your desired wireless network.Enter the password and touch CONNECT.\n\n\n\n\xa0\n\xa0\n'),
 Document(metadata={'line_number': 4, 'source': 'https://dev228679.service-now.com'}, page_content='\nshort_description:Using the SmartView camera view to view inside\t\t\tSmartCool refridgerator\ntext:\n\n\nTouch View Inside.You can label each food with expiration dates. Drag a label from the top and place it on an item.To label the food name and a reminder date, touch &#43;.Enter names using the keyboard.Set a reminder date using the dropdown menu.Touch SAVE.To access the Shopping List, touch the MENU button.Touch Go to Shopping List.\n\n\n\n\xa0\n\xa0\n'),
 Document(metadata={'li

In [10]:
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(),
    vectorstore.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True
)

In [12]:
chat_history = []

query = "You are a customer service agent. Draft an email to a customer on how to use the SmartView Camera."
result = qa_chain({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

/var/folders/bt/6jjk1bl542q287v3ww2df2n00000gn/T/ipykernel_24786/1641145281.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({'question': query, 'chat_history': chat_history})


Answer: Subject: Guide on Using the SmartView Camera Feature in Your SmartCool Refrigerator

Dear Valued Customer,

I hope this email finds you well. We appreciate your interest in utilizing the SmartView camera feature in your SmartCool refrigerator. Here is a step-by-step guide to help you effectively use this feature:

1. Touch View Inside on the SmartCool fridge display screen.
2. You can label each food item with expiration dates by dragging a label from the top and placing it on the item.
3. For food names and reminder dates, touch the "+" symbol.
4. Enter the names using the keyboard, set a reminder date using the dropdown menu, and touch SAVE to confirm.
5. To access the Shopping List, touch the MENU button on the display screen and then select Go to Shopping List.

By following these steps, you will be able to make the most out of the SmartView camera feature in your SmartCool refrigerator.

If you encounter any difficulties or have any further questions, please do not hesitat